# Imports

In [5]:
import os
import re
from typing import List, Dict
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import zipfile
from pathlib import Path
import pandas as pd

# Acesso a API
- Identificação e download dos arquivos ZIP selecionados 
- Extração dos ZIPs 
- Identificação dos arquivos com Despesas com Eventos/Sinistros

### Identificação e download dos arquivos ZIP selecionados 

In [ ]:
def get_years(URL: str, session: requests.Session) -> List[str]:
    """
    Obtém a lista de anos disponíveis na página base.
    """
    resp = session.get(URL, timeout=30)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")

    padrao_ano = re.compile(r"^(20\d{2})/$")

    anos = []
    for link in soup.find_all("a"):
        href = link.get("href", "")
        match = padrao_ano.match(href)
        if match:
            anos.append(match.group(1))

    if not anos:
        raise RuntimeError("Nenhum ano encontrado no diretório base.")

    return anos

def get_trimesters(URL: str, ano: str, session: requests.Session) -> List[Dict[str, any]]:
    url_ano = urljoin(URL, f"{ano}/")
    print(f"\nProcessando ano {ano}...")
    resp_ano = session.get(url_ano, timeout=30)
    if resp_ano.status_code != 200:
        print(f"Falha ao acessar {url_ano}")
        return []

    soup_ano = BeautifulSoup(resp_ano.text, "html.parser")
    arquivos = []

    # Para cada ZIP
    for link in soup_ano.find_all("a"):
        href = link.get("href", "")
            
        if not href.lower().endswith(".zip"):
                continue
            
        trimestre = extract_trimesters(href)

        arquivos.append({
            "ano": ano,
            "trimestre": trimestre,
            "nome": href,
            "url": urljoin(url_ano, href)
        })

    if not arquivos:
        print(f"Nenhum ZIP encontrado para {ano}")
        return []

    # Selecionar os 3 ultimos semestres
    arquivos.sort(key=lambda x: x["trimestre"], reverse=True)
    return arquivos[:3]

def extract_trimesters(nome_arquivo: str) -> int:
    """
    Extrai os trimestre dos nomes de arquivos ZIP.
    """
    nome = nome_arquivo.lower()

    # Encontrar trimestre no nome
    padroes_trimestre = [
        r'([1-4])\s*t',              # 1T, 1t
        r'([1-4])\s*trim',           # 1trim
        r'([1-4])\s*trimestre',      # 1trimestre
        r'([1-4])\s*[-_ ]\s*trim',   # 1-trim
    ]

    trimestre = None
    for padrao in padroes_trimestre:
        match = re.search(padrao, nome)
        if match:
            trimestre = int(match.group(1))
            break

    if trimestre is None:
        raise ValueError(f"Trimestre não encontrado no nome do arquivo: {nome_arquivo}")

    return trimestre

def download_trimesters(pasta_destino, session, ano, trimestres):
    pasta_ano = os.path.join(pasta_destino, ano)
    os.makedirs(pasta_ano, exist_ok=True)

    for arq in trimestres:
        destino = os.path.join(pasta_ano, arq["nome"])

        if os.path.exists(destino):
            print(f"Já existe: {arq['nome']}")
            continue

        print(f"Baixando: {arq['nome']}")

        with session.get(arq["url"], stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(destino, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

def baixar_ultimos_3_trimestres(pasta_destino: str):
    URL = "https://dadosabertos.ans.gov.br/FTP/PDA/demonstracoes_contabeis/"
    os.makedirs(pasta_destino, exist_ok=True)

    session = requests.Session()
    session.headers.update({"User-Agent": "ANS-Crawler/1.0"})

    # 1- Acessar lista de anos
    anos = get_years(URL, session)

    # 2- Para cada ano obter trimestres
    for ano in anos:
        trimestres = get_trimesters(URL, ano, session)

        # 3- Baixar os trimestres selecionados
        download_trimesters(pasta_destino, session, ano, trimestres)

In [ ]:
baixar_ultimos_3_trimestres(pasta_destino= "downloads")

### Extração dos ZIPs

In [57]:
def extrair_zip(caminho_zip, pasta_destino):
    """ 
    Extrai um aquivo ZIP para uma pasta destino
    """
    pasta_destino = Path(pasta_destino)
    pasta_destino.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(caminho_zip, "r") as zip_ref:
        zip_ref.extractall(pasta_destino)

def extrair_todos_os_zips(pasta_origem, pasta_destino):
    """ 
    Extrai todos os ZIPs de uma pasta e coloca os arq extraidos para a pasta detino
    """
    pasta_origem = Path(pasta_origem)
    pasta_destino = Path(pasta_destino)

    print(f"Extraindo pasta {pasta_origem}")

    pasta_destino.mkdir(parents=True, exist_ok=True)

    for zip_path in pasta_origem.rglob("*.zip"):
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(pasta_destino)

In [58]:
for i in range(2007, 2026):
    extrair_todos_os_zips(f"downloads/{i}/", f"downloads/{i}/extraido")

Extraindo pasta downloads\2007
Extraindo pasta downloads\2008
Extraindo pasta downloads\2009
Extraindo pasta downloads\2010
Extraindo pasta downloads\2011
Extraindo pasta downloads\2012
Extraindo pasta downloads\2013
Extraindo pasta downloads\2014
Extraindo pasta downloads\2015
Extraindo pasta downloads\2016
Extraindo pasta downloads\2017
Extraindo pasta downloads\2018
Extraindo pasta downloads\2019
Extraindo pasta downloads\2020
Extraindo pasta downloads\2021
Extraindo pasta downloads\2022
Extraindo pasta downloads\2023
Extraindo pasta downloads\2024
Extraindo pasta downloads\2025
